# Tree Simulation
With R kernel

In [3]:
install.packages("arulesCBA")


Die heruntergeladenen Bin"arpakete sind in 
	/var/folders/kj/dkjqkk2n3wq2zfbttgdpjrj80000gn/T//RtmpyMpQOZ/downloaded_packages


In [4]:
list_of_packages <- c ("synthpop", "insight", "party", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma")

lapply(list_of_packages, FUN= function(X){
  do.call("require", list(X))
})

Lade n"otiges Paket: synthpop



Find out more at https://www.synthpop.org.uk/

Lade n"otiges Paket: insight

Lade n"otiges Paket: party

Lade n"otiges Paket: grid

Lade n"otiges Paket: mvtnorm

Lade n"otiges Paket: modeltools

Lade n"otiges Paket: stats4

Lade n"otiges Paket: strucchange

Lade n"otiges Paket: zoo


Attache Paket: 'zoo'


Die folgenden Objekte sind maskiert von 'package:base':

    as.Date, as.Date.numeric


Lade n"otiges Paket: sandwich

Lade n"otiges Paket: dplyr


Attache Paket: 'dplyr'


Das folgende Objekt ist maskiert 'package:party':

    where


Die folgenden Objekte sind maskiert von 'package:stats':

    filter, lag


Die folgenden Objekte sind maskiert von 'package:base':

    intersect, setdiff, setequal, union


Lade n"otiges Paket: rpart

Lade n"otiges Paket: rpart.plot

Warning message:
"Paket 'rpart.plot' wurde unter R Version 4.3.2 erstellt"
Lade n"otiges Paket: randomForest

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attache Paket: 'randomForest'


D

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

In [29]:
load("/Users/emmafoessing/Documents/Master/MA/Synthpop/cpspop.RData")
adult <- read.csv("/Users/emmafoessing/Documents/Master/MA/Code/adult_preprocessed.csv")
# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

Ein Datensatz zu genieren (m = 1) ist ausreichend, da ich keine Varianzanalyse machen werde. Damit die Ergebnisse nicht von einem zufälligen Prozess abhängen ist es sinnvoll über ein paar runs Mittelwerte zu bilden (50–100)

In [6]:
## tree fct with control component

build_tree <- function(data,trainsplit = 0.8, cp = 0.01){ #minsplit=20, minbucket=5, maxdepth=20, 
    train_index <- sample(1:nrow(data), nrow(data)*trainsplit)
    # train dataset formation
    train_set <- data[train_index, ]
    # test dataset formation
    test_set <- data[-train_index, ]

    tree <- rpart(income ~ ., data = train_set, cp=cp)

    # Predict on the test set
    predictions <- predict(tree, test_set)#, type = "prob")

    # for factored variables this will give probabilities, so there is a need to create the actual predictions
    if (is.factor(data$income)){
        # Initialize predictions as just the probability predictions
        predictions_prob <- predictions
        predictions <- list(probabilities = predictions_prob)
        preds <- apply(predictions_prob, 1, function(row) {
            # Get the index of the max value in the row
            max_index <- which.max(row)
            # Return the column name using the index
            return(colnames(predictions_prob)[max_index])
        })
        # Add actual predictions to the predictions list
        predictions$classes <- as.factor(preds)
    }

    # plot the tree
    #rpart.plot(tree)

    # Results
    results <- list(train_set, test_set, tree, predictions)
    names(results) <- c("train_set", "test_set", "tree", "predictions")
    return(results)
}

In [7]:
## Calculate evaluation metrics for continuous target variable income

evaluation_cont_single <- function(predictions, test_set){
  MAE <- mean(abs(predictions - test_set$income))
  MSE <- mean((predictions - test_set$income)^2)
  RMSE <- sqrt(MSE)

  SS_res <- sum((test_set$income - predictions)^2)
  SS_tot <- sum((test_set$income - mean(test_set$income))^2)
  R_squared <- 1 - (SS_res / SS_tot)

  MAPE <- mean(abs((test_set$income - predictions) / test_set$income)) * 100

  # Create a data frame for the metrics
  evaluation_table <- data.frame(
    Metric = c("Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "Root Mean Squared Error (RMSE)",
              "R-squared (R²)", "Mean Absolute Percentage Error (MAPE)"),
    Value = c(MAE, MSE, RMSE, R_squared, MAPE)
  )

  # Format the values in the Value column to display without scientific notation
  evaluation_table$Value <- as.numeric(format(evaluation_table$Value, scientific = FALSE))

  # Print the evaluation table
  return(evaluation_table)
}

## function for comparison

evaluation_metrics_cont <- function(predictions_orig, test_set_orig, predictions_syn, test_set_syn){

  # create table for original data
  eval_orig <- evaluation_cont_single(predictions_orig, test_set_orig)
  # create table for synthetic data
  eval_syn <- evaluation_cont_single(predictions_syn, test_set_syn)

  # Merge the two dataframes and set the first column as row names
  metrics_df <- merge(eval_orig, eval_syn, by = "Metric", suffixes = c("_orig", "_syn"))
   # Set "Metric" column as row names
  rownames(metrics_df) <- metrics_df$Metric
  
  # Remove "Metric" column
  metrics_df <- metrics_df[, -1]
  
  # Calculate the numeric difference
  metrics_df$Difference <- metrics_df$Value_syn - metrics_df$Value_orig

  return(metrics_df)
}


In [8]:
## Calculate evaluation metrics for factored targets
evaluation_metrics_factor <- function(predictions_orig, test_set_orig, predictions_syn, test_set_syn){
    # confusion matrix for the prediction on original data
    cm_orig <- confusionMatrix(predictions_orig, test_set_orig$income,
                mode = "everything")
    # confusion matrix for the prediction on synthetic data         
    cm_syn <- confusionMatrix(predictions_syn, test_set_syn$income,
                mode = "everything")

    # saving evaluation metrics
    accuracy_orig <- cm_orig$overall['Accuracy']
    f1_orig       <- cm_orig$byClass['F1']
    sens_orig     <- cm_orig$byClass['Sensitivity']
    spec_orig     <- cm_orig$byClass['Specificity']
    accuracy_syn  <- cm_syn$overall['Accuracy']
    f1_syn        <- cm_syn$byClass['F1']
    sens_syn      <- cm_syn$byClass['Sensitivity']
    spec_syn      <- cm_syn$byClass['Specificity']

    # Saving the differences
    accuracy_diff <- accuracy_syn - accuracy_orig
    f1_diff <- f1_syn - f1_orig
    sens_diff <- sens_syn - sens_orig
    spec_diff <- spec_syn - spec_orig

    # Create the dataframe
    metrics_df <- data.frame(
    Metric = c("Accuracy", "F1", "Sensitivity", "Specificity"),
    Original = c(accuracy_orig, f1_orig, sens_orig, spec_orig),
    Synthetic = c(accuracy_syn, f1_syn, sens_syn, spec_syn),
    Difference = c(accuracy_diff, f1_diff, sens_diff, spec_diff)
    )

    return(metrics_df)
}

To Do:
- Für die Originaldaten wird es am Ende reichen nur einmal auf dem gleichen Seed die Prediction zu machen --> diesen Loop genau so, aber nicht wieder bei jedem Modell
- Irgendwie einbauen den complexity parameter zu speichern
- ggf. in der simulation nur alles einmal und den vergleich der daten (syn–orig) dann im nachhinein --> eine simulation fct für die originaldaten und eine für alle synthetischen? --> generieren der synthetischen daten weicht aber ab--> in functionsargumenten festlegen welche und diese wird dann durchlaufen? oder doch eine simulation mit allen methoden --> evtl. am ende --> würde echt lange dauern

In [76]:
simulation <- function(data, method = "cart", k = nrow(data), nrun = 5, k_fold = 10, steps = 10){

  # create array to save the synthetic data
  syn_data <- array(data = NA, dim = c(k, ncol(data), nrun)) #, dimnames = list("row", "colum", "run"))
  # create empty vectors to safe trees
  tree_orig <- list()
  tree_syn <- list()
  # create vectors to safe loss
  loss_orig <- rep(0, steps)
  loss_syn <- rep(0, steps)
  # create empty array to store cp values chosen
  cp_vals <- array(data = NA, dim = c(nrun, 2))
  # create empty list to store evaluation dataframes
  eval_list <- list()

  # for loss-calculation factored variables need to be converted to numeric
  if (is.factor(data$income)) {
    data$income <- as.factor(as.numeric(data$income == ">50K" ))
  }

  s <- 1234
  for (i in 1:nrun){
    # vary seed with each run
    s <- s + 1

    # generate synthetic data
    gen_data <- syn(data = data, method = method, k = k, seed = s)
    # save the data from each run
    syn_data[,,i] <- as.matrix(gen_data$syn)

    # for each fold add up loss, choose the parameter for which the loss is smallest
    # loop over different cp values?? It is pretty obv that loss will just get smaller with smaller cp values

    complexity <- seq(0.0001, 0.01, length.out = steps)
    
    # Randomly split the data set into k-subsets (or k-fold)
    data_orig <- as.data.frame(data)
    data_syn <- as.data.frame(gen_data$syn)
    datalist_orig <- split(data_orig, sample(1:k_fold, nrow(data_orig), replace=T)) #list of k same-sized elements that are slices of the data
    datalist_syn <- split(data_syn, sample(1:k_fold, nrow(data_syn), replace=T)) #list of k same-sized elements that are slices of the data

    # leave-one-out CV for prediction
    for (j in 1:k_fold) {
      #split data in k folds
      data_val_orig <- datalist_orig[[j]]    #j-th of the k folds, validation set
      data_train_orig <- datalist_orig[-j]   #rest of the data without j-th of the k folds, training set
      data_train_orig <- bind_rows(data_train_orig) #convert list to dataframe
      data_val_syn <- datalist_syn[[j]]    #j-th of the k folds, validation set
      data_train_syn <- datalist_syn[-j]   #rest of the data without j-th of the k folds, training set
      data_train_syn <- bind_rows(data_train_syn) #convert list to dataframe

      # optional parameters to prevent overfitting: minbucket, minsplit, maxdepth
      for (l in 1:length(complexity)){
        # create income prediction with orig. data
        tree_orig[[l]] <- rpart(income ~ ., data = data_train_orig, cp = complexity[l])
        # create income prediction with syn. data
        tree_syn[[l]] <- rpart(income ~ ., data = data_train_syn, cp = complexity[l])
        
        # Predict on the test set
        predictions_orig <- predict(tree_orig[l], data_val_orig)
        predictions_syn <- predict(tree_syn[l], data_val_syn)
        print("dim predictions")
        print(dim(predictions_orig))
        #print(class(predictions_orig))
        #print(predictions_orig)

        # safe some loss information and sum over the k-fold loops
        if (is.numeric(data$income)) {
          # Mean Squared Error
          loss_orig[l] <- loss_orig[l] + mean((predictions_orig - tree_orig[[l]]$test_set$income)^2)
          loss_syn[l] <- loss_syn[l] + mean((predictions_syn - tree_syn[[l]]$test_set$income)^2)
        }
        else if (is.factor(data$income)) {
          # Cross-Entropy Loss
          epsilon <- 1e-15  # to prevent log(0) which is undefined
          predicted_probs <- pmax(pmin(predictions_orig[,2], 1 - epsilon), epsilon)
          n <- length(predicted_probs)
          loss_orig[l] <- loss_orig[l] + (-sum(tree_orig[[l]]$test_set$income * log(predicted_probs) + (1 - tree_orig[[l]]$test_set$income) * log(1 - predicted_probs)) / n)
          predicted_probs <- pmax(pmin(predictions_syn, 1 - epsilon), epsilon)
          loss_syn[l] <- loss_syn[l] + (-sum(tree_syn[[l]]$test_set$income * log(predicted_probs) + (1 - tree_syn[[l]]$test_set$income) * log(1 - predicted_probs)) / n)
        }
        else {
          break("The predicted target has to be numeric or factor.")
        }
      }
    }
    # safe only the tree with the minimal loss
    min_orig  <- which(loss_orig==min(loss_orig), arr.ind = TRUE)
    min_syn  <- which(loss_syn==min(loss_syn), arr.ind = TRUE)
    # safe the cp value for the tree with the minimal loss
    cp_vals[i,] <- c(complexity[min_orig], complexity[min_syn])

    train_index_orig <- sample(1:nrow(data), nrow(data)*trainsplit)
    # train dataset formation
    train_set_orig <- data[train_index_orig, ]
    # test dataset formation
    test_set_orig <- data[-train_index_orig, ]

    train_index_syn <- sample(1:nrow(gen_data$syn), nrow(gen_data$syn)*trainsplit)
    # train dataset formation
    train_set_syn <- gen_data$syn[train_index_syn, ]
    # test dataset formation
    test_set_syn <- gen_data$syn[-train_index_syn, ]

    tree_o <- build_tree(data = data, cp = complexity[min_orig])
    tree_s <- build_tree(data = data, cp = complexity[min_syn])
    
    # evaluate the difference
    if (is.numeric(data$income)) {
      eval <- as.data.frame(evaluation_metrics_cont(predictions_orig, test_set_orig, predictions_syn, test_set_syn))
    }
    else if (is.factor(data$income)) {
      predictions_orig <- apply(predictions_orig, 1, function(row) {
            # Get the index of the max value in the row
            max_index <- which.max(row)
            # Return the column name using the index
            return(colnames(predictions_orig)[max_index])
            })
      predictions_syn <- apply(predictions_syn, 1, function(row) {
            # Get the index of the max value in the row
            max_index <- which.max(row)
            # Return the column name using the index
            return(colnames(predictions_syn)[max_index])
            })
      eval <- as.data.frame(evaluation_metrics_factor(predictions_orig, test_set_orig, predictions_syn, test_set_syn))
    }
    else {
       break("The predicted target has to be numeric or factor.")
    }

    eval_list[[i]] <- eval

    print(c("run", i, "completed"))
  }
  
  # now average all dataframes in eval_list
  sum_df <- Reduce(function(x, y) Map(`+`, x, y), eval_list)
  eval_avg <- lapply(sum_df, function(col) col / length(eval_list))

  # Convert the list back to a dataframe
  # Store row names
  rownames <- row.names(eval_list[[1]])

  # Convert the list back to a dataframe
  eval_avg <- as.data.frame(eval_avg)

  # Set back the row names
  row.names(eval_avg) <- rownames

  # returns
  results <- list(eval_avg = eval_avg, syn_data = syn_data, cp_vals = cp_vals)
  return(results)
  
}

In [79]:
?rpart.control

rpart.control              package:rpart               R Documentation

_C_o_n_t_r_o_l _f_o_r _R_p_a_r_t _F_i_t_s

_D_e_s_c_r_i_p_t_i_o_n:

     Various parameters that control aspects of the 'rpart' fit.

_U_s_a_g_e:

     rpart.control(minsplit = 20, minbucket = round(minsplit/3), cp = 0.01, 
                   maxcompete = 4, maxsurrogate = 5, usesurrogate = 2, xval = 10,
                   surrogatestyle = 0, maxdepth = 30, ...)
     
_A_r_g_u_m_e_n_t_s:

minsplit: the minimum number of observations that must exist in a node
          in order for a split to be attempted.

minbucket: the minimum number of observations in any terminal '<leaf>'
          node.  If only one of 'minbucket' or 'minsplit' is specified,
          the code either sets 'minsplit' to 'minbucket*3' or
          'minbucket' to 'minsplit/3', as appropriate.

      cp: complexity parameter.  Any split that does not decrease the
          overall lack of fit by a factor

In [77]:
adult_res <- simulation(adult, nrun = 2, k_fold = 5, steps = 10)


Synthesis
-----------
 age workclass fnlwgt education marital_status occupation relationship race sex capital_gain
 capital_loss hours_per_week native_country income
[1] "dim predictions"
NULL


ERROR: Error in predictions_orig[, 2]: falsche Anzahl von Dimensionen


In [45]:
cps_res <- simulation(cpspop, nrun = 20, k_fold = 5, steps = 10)


Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "1"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "2"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "3"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "4"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "5"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "6"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "7"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "8"         "completed"

Synthesis
-----------
 tax income csp age educ marital race sex ss
[1] "run"       "9"         "completed"

Synthesis
-----------
 tax 

In [46]:
cps_res$cp_vals

0.0023,0.0023
0.0023,0.0023
0.0023,0.0023
0.0012,0.0012
0.0012,0.0012
0.0012,0.0012
0.0012,0.0012
0.0012,0.0012
0.0012,0.0012
0.0012,0.0012
0.0012,0.0012


In [47]:
cps_res$eval_avg

,Value_orig,Value_syn,Difference
,<dbl>,<dbl>,<dbl>
Mean Absolute Error (MAE),2.611207e+04,2.603615e+04,-7.591627e+01
Mean Absolute Percentage Error (MAPE),4.492630e+03,3.530684e+03,-9.619463e+02
Mean Squared Error (MSE),1.675107e+09,1.686016e+09,1.090891e+07
R-squared (R<U+00B2>),3.066210e-01,3.114957e-01,4.874735e-03
Root Mean Squared Error (RMSE),4.091698e+04,4.105264e+04,1.356675e+02


In [ ]:
adult_res$cp_vals

In [ ]:
adult_res$eval_avg

In [ ]:
#Saving the data:
saveRDS(cps_res, file = (paste0(here(), "/simulation/cps_CART_res.RData")))
saveRDS(adult_res, file = (paste0(here(), "/simulation/adult_CART_res.RData")))